In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip uninstall keras
!pip install "keras==2.9.0rc0"

Found existing installation: keras 2.9.0
Uninstalling keras-2.9.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/keras-2.9.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/keras/*
Proceed (y/n)? ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 14.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0


In [3]:
from os import listdir
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import keras
from keras.applications.vgg19 import VGG19

from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import random
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
# import tensorflow.keras
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

In [4]:



raw_folder = "/content/drive/MyDrive/ai/data"
def save_data(raw_folder=raw_folder):

    dest_size = (128, 128)
    print("Bắt đầu xử lý ảnh...")

    pixels = []
    labels = []

    # Lặp qua các folder con trong thư mục raw
    for folder in listdir(raw_folder):
        if folder!='.DS_Store':
            print("Folder=",folder)
            # Lặp qua các file trong từng thư mục chứa các em
            for file in listdir(raw_folder  +'/' + folder):

                if file!='.DS_Store' and (random.random() <=0.333):
                    print("File=", file)
                    pixels.append( cv2.resize(cv2.imread(raw_folder  +'/'+ folder +"/" + file),dsize=(128,128)))
                    labels.append( folder)

    pixels = np.array(pixels)
    labels = np.array(labels)#.reshape(-1,1)

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)
    print(labels)

    file = open( '/content/drive/MyDrive/ai/pix.data', 'wb')
    # dump information to that file
    pickle.dump((pixels,labels), file)
    # close the file
    file.close()

    return

def load_data():
    file = open("/content/drive/MyDrive/ai/" +'pix.data', 'rb')

    # dump information to that file
    (pixels, labels) = pickle.load(file)

    # close the file
    file.close()

    print(pixels.shape)
    print(labels.shape)
    print(labels)


    return pixels, labels

# save_data()

X,y = load_data()
#random.shuffle(X)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=100)

print(X_train.shape)
print(y_train.shape)


model = keras.Sequential()

model.add(Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv1', input_shape=(128, 128, 3)))
model.add(Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block1_maxpool'))

model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv1'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block2_maxpool'))

model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv1'))
model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv2'))
model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv3'))
model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv4'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block3_maxpool'))

model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv1'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv2'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv3'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv4'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block4_maxpool'))

model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv1'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv2'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv3'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv4'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block5_maxpool'))

model.add(Flatten())
model.add(Dense(1024, activation='relu', name='fc1'))
model.add(Dense(1024, activation='relu', name='fc2'))
model.add(Dropout(0.5))

model.add(Dense(28, activation='softmax', name='predictions'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



model.load_weights("/content/drive/MyDrive/weights-20-0.97.hdf5")
filepath="/content/drive/MyDrive/weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
    rescale=1./255,
	width_shift_range=0.1,
    height_shift_range=0.1,
	horizontal_flip=False,
    brightness_range=[0.2,1.5], fill_mode="nearest")

aug_val = ImageDataGenerator(rescale=1./255)

modelhist=model.fit_generator(aug.flow(X_train, y_train, batch_size=128),
                              #  X_train, y_train, batch_size=8,
                               epochs=20,# steps_per_epoch=len(X_train)//64,
                                validation_data=aug.flow(X_test,y_test,
                                batch_size=128),
                              # validation_data = ,
                               callbacks=callbacks_list)

model.save("model.h5")

(28029, 128, 128, 3)
(28029, 28)
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]
(22423, 128, 128, 3)
(22423, 28)


<ipython-input-4-cf3fcc814f41>:118: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  modelhist=model.fit_generator(aug.flow(X_train, y_train, batch_size=128),


Epoch 1/20
176/176 [==============================] - ETA: 0s - loss: 0.0110 - accuracy: 0.9585
Epoch 1: val_accuracy improved from -inf to 0.98787, saving model to /content/drive/MyDrive/weights-01-0.99.hdf5
176/176 [==============================] - 179s 920ms/step - loss: 0.0110 - accuracy: 0.9585 - val_loss: 0.0031 - val_accuracy: 0.9879
Epoch 2/20
176/176 [==============================] - ETA: 0s - loss: 0.0059 - accuracy: 0.9777
Epoch 2: val_accuracy did not improve from 0.98787
176/176 [==============================] - 154s 874ms/step - loss: 0.0059 - accuracy: 0.9777 - val_loss: 0.0028 - val_accuracy: 0.9870
Epoch 3/20
176/176 [==============================] - ETA: 0s - loss: 0.0044 - accuracy: 0.9827
Epoch 3: val_accuracy did not improve from 0.98787
176/176 [==============================] - 153s 870ms/step - loss: 0.0044 - accuracy: 0.9827 - val_loss: 0.0043 - val_accuracy: 0.9797
Epoch 4/20
176/176 [==============================] - ETA: 0s - loss: 0.0035 - accuracy: 0.9

KeyboardInterrupt: ignored

In [5]:

model.evaluate(X_test, y_test)

176/176 [==============================] - 14s 63ms/step - loss: 0.0562 - accuracy: 0.9966


[0.05620690807700157, 0.9966107606887817]